In [1]:
%cleanup -f

In [114]:
%%configure -f
{
    "numExecutors": 12,
    "driverMemory": "20g"
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1673908281065_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1673908281065_0003,pyspark,idle,Link,Link,None,✔


In [178]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.ml.feature import OneHotEncoder, Normalizer, MinMaxScaler, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector, Vectors, VectorUDT
from pyspark.ml import Pipeline
import numpy as np



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [206]:
@f.udf(returnType=StringType())
def resolve_sector_name(cell_name):
    """
    resolve cell name to match with EDS event data
    """
    try:
        split_parts = cell_name.split("_")
        m2000id = split_parts[0]
        site_name = split_parts[1]
        cell_number = split_parts[3]
        return f"{m2000id}_{site_name}_{cell_number}"
    except Exception as _:
        return cell_name
@f.udf(returnType=VectorUDT())
def sparse_vector_sum(vectors):
    res = None
    for vec in vectors:
        if res is None:
            res = vec
        else:
            res = np.add(vec, res)
    return SparseVector(len(res), {k: v for k, v in enumerate(res) if v != 0})

@f.udf(returnType=ArrayType(FloatType()))
def to_dense(sparse_vector):
    v = DenseVector(sparse_vector)
    feature = [float(x) for x in v]
    feature.append(0.0)  # adding util placeholder
    return feature

@f.udf(returnType=ArrayType(FloatType()))
def to_dense_flat(sparse_vector):
    v = DenseVector(sparse_vector)
    feature = [float(x) for x in v]
    return feature

@f.udf(returnType=ArrayType(ArrayType(StringType())))
def construct_nodes(edge_input):
    """
    node types:
    0 - user
    1 - cell
    2 - sector
    """
    node_list = []
    for a in edge_input:
        if [a[0], 0] not in node_list:
            node_list.append([a[0], 0])
        if [a[1], 1] not in node_list:
            node_list.append([a[1], 1])
        if [a[2], 2] not in node_list:
            node_list.append([a[2], 2])
    return node_list

@f.udf(returnType=VectorUDT())
def add_sparse31(v1, val):
    v1d = dict(zip(v1.indices, v1.values))
    new_idx = v1.size
    v1d[new_idx] = val
    return SparseVector(new_idx+1, {k: v for k, v in v1d.items() if v != 0.0})

@f.udf(returnType=ArrayType(ArrayType(StringType())))
def construct_edge_index(edge_input):

    edge_index = []
    for a in edge_input:
        if [a[0], a[1]] not in edge_index:
            edge_index.append([a[0], a[1]])
    return edge_index

@f.udf(returnType=ArrayType(ArrayType(IntegerType())))
def construct_node_types(node_list):
    node_types = [[] for i in range(2)]
    for ntype, nid in node_list:
        node_types[int(ntype)].append(nid)
    return node_types
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
# Event Data
eventdf = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/tokenised_ebmr_sample/dt=2023-01-10/')

# CEI Data
cei_df = spark.read.parquet('s3://kl-prod-bi-analy-data-extract/features/kl-prod/Mobility/customer_exp_index_v3/part_dt=2023-01-11/')
cei_df =cei_df.withColumnRenamed('subs_id', 'Subs_Id')

# Cell INFO
@f.udf(returnType=StringType())
def resolve_cell_name(cell_name):
    """
    resolve cell name to match with EDS event data
    """
    try:
        split_parts = cell_name.split("_")
        m2000id = split_parts[0]
        site_name = split_parts[1]
        technology = split_parts[3]
        cell_number = split_parts[4]
        return f"{m2000id}_{site_name}_{technology}_{cell_number}"
    except Exception as _:
        return cell_name

locglbl = spark.read.parquet(
    's3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/EDS_locglbl_v3_extract/part_dt=2023-01-11/'
)
cell_lookup = spark.read.parquet(
    's3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/TD_cell_look_up/part_dt=2023-01-11/'
)
site_df = spark.read.parquet(
    's3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/TD_site_table/part_dt=2023-01-11/'
)

cell_info = (
    locglbl.join(
        cell_lookup.filter(
            f.col("Radio_Type_Cd").isin(["L", "U", "N"])
        ),  # ignore 3G cells. ie. 'U'. Ignores 5G for test. ie 'N'
        on=locglbl.glbl_loc_id == cell_lookup.Glbl_loc_Id,
        how="inner",
    )
    .withColumn("m2000_id", f.split(f.col("Cell_Name"), "_").getItem(0))
    .join(
        site_df,  # .filter(f.col("sa6_state_nm") == sa6_nm),
        on="m2000_id",
        how="inner",
    )
    .withColumn("cell_name_adjusted", resolve_cell_name(f.col("Cell_Name")))
    .select(
        locglbl.glbl_loc,
        locglbl.glbl_loc_id,
        f.col("cell_name_adjusted").alias("cell_name"),
        locglbl.mcc,
        locglbl.mnc,
        site_df.state
    )
)
cell_info

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[glbl_loc: string, glbl_loc_id: string, cell_name: string, mcc: string, mnc: string, state: string]

In [119]:
cell_info = cell_info.withColumn('m2000id', f.split('cell_name', '_').getItem(0).cast('string'))
cell_info = cell_info.withColumn('sector_name', resolve_sector_name(f.col('cell_name')))
cell_info.persist()
cell_info.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------+--------------------------+-----------------------------+---+---+---------------+-------+------------------------+
|glbl_loc                  |glbl_loc_id               |cell_name                    |mcc|mnc|state          |m2000id|sector_name             |
+--------------------------+--------------------------+-----------------------------+---+---+---------------+-------+------------------------+
|U_505_03_00345__7264_     |U_505_03_00345_07264_07264|303455_MillPark_U21B_1       |505|03 |Victoria       |303455 |303455_MillPark_1       |
|L_505_03__20282__006112278|L_505_03_20282_|_006112278|201593_CampsieNorth_L18B_2   |505|03 |New South Wales|201593 |201593_CampsieNorth_2   |
|L_505_03__30345__8471850  |L_505_03_30345_|_008471850|338598_Greensborough_L21H_2  |505|03 |Victoria       |338598 |338598_Greensborough_2  |
|L_505_03__30338__7707966  |L_505_03_30338_|_007707966|303039_DocklandsStadm_L18A_12|505|03 |Victoria       |303039 |303039_DocklandsStadm_12|

In [120]:
@f.udf(returnType=IntegerType())
def is_regular(e):
    if int(e) in [13, 7, 22, 26, 30, 23, 24, 25]:
        return 1
    else:
        return 0
@f.udf(returnType=IntegerType())
def is_irregular(e):
    if int(e) in [13, 7, 22, 26, 30, 23, 24, 25]:
        return 0
    else:
        return 1

eventdf = eventdf.withColumn('hour', f.hour('event_timestamp'))
eventdf = eventdf.withColumn(
    'regular_event', is_regular(f.col('event_type_cd'))
).withColumn(
    'irregular_event', is_irregular(f.col('event_type_cd'))
)
eventdf.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+----+-------------+---------------+
|    event_timestamp|event_type_cd|result|            loc_info|     TAC|             Subs_Id|prefix|matched|hour|regular_event|irregular_event|
+-------------------+-------------+------+--------------------+--------+--------------------+------+-------+----+-------------+---------------+
|2023-01-10 10:14:09|           15|     0|U_505_03_336__16283_|14869004|063d6b4f880e925a9...|   118|  false|  10|            0|              1|
|2023-01-10 20:20:39|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|  20|            0|              1|
|2023-01-10 13:23:19|           15|     0|  U_505_03_336__840_|14869004|063d6b4f880e925a9...|   118|  false|  13|            0|              1|
|2023-01-10 00:46:22|           15|     0|  U_505_03_336__640_|14869004|063d6b4f880e925a9...|   118|  false|   0|            0|         

In [121]:
# site timezone
site_tz = spark.read.csv('s3://kl-prod-bi-analy-data-extract/data/kl-prod/site_timezone/part_dt=2023-01-11/', header=True)
site_tz.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+--------+
|M2000_ID|State          |timezone|
+--------+---------------+--------+
|202096  |New South Wales|null    |
|444470  |Queensland     |null    |
|202670  |New South Wales|null    |
|404682  |Queensland     |null    |
|220130  |New South Wales|null    |
+--------+---------------+--------+
only showing top 5 rows

In [122]:
utilisation = spark.read.parquet('s3://kl-prod-bi-analy-data-extract/features/kl-prod/ICE/RAN_util_hr_multibeam/part_dt=2023-01-11/')
utilisation = utilisation.withColumn('hour', f.hour('starttime'))
utilisation.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------------+---------+--------+------------------+------------------------+----------------------------------+------------------+----------------------+--------------------+----------------------+-------------+-------------+-----------+-----------+----+
|starttime          |sector_name             |multibeam|m2000_id|prb_dl_util_pct   |weighted_prb_dl_util_pct|prb_dl_util_pct_multibeam_weighted|utilisation       |chmeas_prb_dl_used_avg|mean_prb_avail_pdsch|traf_activeuser_dl_avg|thrp_dl_qci_8|thrp_ul_qci_8|dlpayload  |ulpayload  |hour|
+-------------------+------------------------+---------+--------+------------------+------------------------+----------------------------------+------------------+----------------------+--------------------+----------------------+-------------+-------------+-----------+-----------+----+
|2023-01-10 00:00:00|200027_MLCCentre_1      |0        |200027  |1.5300000000000002|1.5300000000000002      |1.5300000000000002         

## Get Valid Data for a set of sectors

In [ ]:
# flat_event = eventdf.join(
#     cell_info,
#     on=eventdf.loc_info==cell_info.glbl_loc
# ).drop('glbl_loc', 'glbl_loc_id', 'loc_info', 'mcc', 'mnc').join(
#     utilisation,
#     on=['hour', 'sector_name']
# )
# flat_event.persist()
# flat_event.count()

In [ ]:
# flat_event.show()

In [ ]:
# flat_event = flat_event.select(
#     'hour', 'Subs_Id', 'cell_name', 'sector_name', 'event_type_cd', 'regular_event', 'irregular_event', 'utilisation', 'state'
# ).filter('utilisation IS NOT NULL AND Subs_Id IS NOT NULL AND length(Subs_Id) < 10')

# flat_event.write.mode('overwrite').parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/flat_event/')

In [123]:
flat_event = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/flat_event/')
flat_event = flat_event.withColumn(
    'utilisation', f.col('utilisation').cast('int')
)

flat_event.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+--------------------+--------------------+-------------+-------------+---------------+-----------+---------------+
|hour|  Subs_Id|           cell_name|         sector_name|event_type_cd|regular_event|irregular_event|utilisation|          state|
+----+---------+--------------------+--------------------+-------------+-------------+---------------+-----------+---------------+
|  20|354931842|202926_Bronte_L18A_3|     202926_Bronte_3|            7|            1|              0|         88|New South Wales|
|   9|354931842|201522_Waterloo_L...|   201522_Waterloo_1|            8|            0|              1|         21|New South Wales|
|  11|354931842|202926_Bronte_L18A_3|     202926_Bronte_3|           13|            1|              0|         53|New South Wales|
|   9|354931842|201522_Waterloo_L...|   201522_Waterloo_2|           13|            1|              0|         32|New South Wales|
|   6|354931842|227393_Coogee_L08A_3|     227393_Coogee_3|            7|           

In [124]:
flat_event.select(
    'sector_name', 'hour', 'utilisation'
).dropDuplicates().orderBy(f.col('utilisation').desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-----------+
|         sector_name|hour|utilisation|
+--------------------+----+-----------+
|200499_RNSHospital_2|  12|         99|
|200499_RNSHospital_2|  11|         99|
|200499_RNSHospital_2|  10|         99|
|227592_Cabramatta...|  18|         98|
|227592_Cabramatta...|  19|         98|
|   227591_Carramar_1|  16|         98|
|201803_Penhust594...|  13|         98|
|  227705_KingsPark_2|  16|         98|
|  202356_NelsonBay_2|  19|         98|
|201710_LiverpoolN...|  20|         98|
|202803_FairfieldS...|  14|         98|
|202803_FairfieldS...|  20|         98|
|227592_Cabramatta...|  14|         98|
|404574_GladstoneC...|  19|         98|
|202803_FairfieldS...|  15|         98|
|   227591_Carramar_1|  15|         98|
|     477396_Noosa2_1|  14|         98|
|404168_Flindersvi...|  20|         98|
|     477396_Noosa2_1|  13|         98|
|     477396_Noosa2_1|  15|         98|
+--------------------+----+-----------+
only showing top 20 rows

In [125]:
flat_event.select(
    'sector_name', 'hour', 'utilisation'
).dropDuplicates().groupBy('utilisation').count().orderBy(f.col('count').desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
|utilisation|count|
+-----------+-----+
|          3|11050|
|          2|10609|
|          4| 9576|
|          5| 8223|
|          1| 7918|
|          6| 7618|
|          7| 6871|
|          8| 6393|
|          9| 6134|
|         10| 5600|
|         11| 5280|
|         12| 5115|
|         13| 4701|
|         14| 4490|
|         15| 4295|
|         16| 4167|
|         17| 3921|
|         18| 3650|
|         19| 3527|
|         20| 3426|
+-----------+-----+
only showing top 20 rows

In [126]:
flat_event.groupBy('sector_name', 'hour').count().orderBy(f.col('count').desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-----+
|        sector_name|hour|count|
+-------------------+----+-----+
|   201557_Lambton_2|  17|59438|
|   201557_Lambton_2|  20|51498|
|   201557_Lambton_2|  19|51191|
|    227393_Coogee_3|  17|50021|
|   201557_Lambton_2|  18|49817|
|227994_PenrithCBD_3|  11|49808|
|227994_PenrithCBD_3|  12|48072|
|    227393_Coogee_3|  14|47996|
|    227393_Coogee_3|  13|47888|
|202934_Camperdown_3|  16|47851|
|    227393_Coogee_3|  16|47699|
|340194_ClydeNorth_2|  17|47055|
|227994_PenrithCBD_3|  13|46968|
|    227393_Coogee_3|  12|46931|
|340194_ClydeNorth_2|  18|46802|
|227994_PenrithCBD_3|  10|46438|
|    227393_Coogee_3|  15|45627|
|340194_ClydeNorth_2|  16|45312|
|340194_ClydeNorth_2|  19|44827|
|    227393_Coogee_3|  18|44499|
+-------------------+----+-----+
only showing top 20 rows

In [127]:
flat_event.groupBy('sector_name', 'hour').agg(f.countDistinct('Subs_Id').alias('count')).orderBy(f.col('count').desc()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-----+
|         sector_name|hour|count|
+--------------------+----+-----+
|202101_MillersPoi...|  16| 8724|
| 202934_Camperdown_3|  16| 8707|
|227778_TownHallSRA_2|  16| 8062|
|303146_FlindersSt...|  16| 7903|
|202101_MillersPoi...|   7| 7717|
|227572_NorthAuburn_2|  16| 7688|
| 202934_Camperdown_3|   7| 7664|
|200253_CntlTwnHal...|  16| 7282|
|201850_Strawberry...|  16| 6955|
| 202934_Camperdown_3|  15| 6943|
|227572_NorthAuburn_2|  15| 6865|
|227712_DulwichHil...|  16| 6757|
|227778_TownHallSRA_2|   7| 6720|
|303146_FlindersSt...|   7| 6529|
|227480_SurryHills...|  16| 6458|
|    227513_MtLewis_1|  16| 6426|
| 202934_Camperdown_3|  17| 6422|
|200258_MusmCtlTwn...|  16| 6314|
|227572_NorthAuburn_2|  14| 6273|
|227572_NorthAuburn_2|  17| 6262|
+--------------------+----+-----+
only showing top 20 rows

In [128]:
flat_event.groupBy('Subs_Id', 'hour').agg(f.countDistinct('sector_name').alias('count')).orderBy(f.col('count').desc()).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-----+
|  Subs_Id|hour|count|
+---------+----+-----+
|367441676|  14|   74|
|391419389|  10|   70|
|372354612|  11|   67|
|388852765|  11|   67|
|390914401|  10|   66|
|371435688|   8|   66|
|383291285|  11|   65|
|318555605|  12|   64|
|385706256|  11|   63|
|379378503|   6|   63|
|390291923|  20|   62|
|391376089|  20|   62|
|372969865|  17|   62|
|387597406|   6|   62|
|366325375|   5|   62|
|388528039|  15|   61|
|390192219|  10|   61|
|380536297|  14|   61|
|389269282|   9|   61|
|390817038|   5|   61|
+---------+----+-----+
only showing top 20 rows

In [129]:
@f.udf(returnType=IntegerType())
def calc_weighted_utilisation(event_list):
    sector_dict = {}
    for e in event_list:
        if e[1] not in sector_dict.keys():
            sector_dict[e[1]] = {'events': 0, 'util': 0}
        sector_dict[e[1]]['events'] += e[0]
        sector_dict[e[1]]['util'] = e[2]
    
    total = sum([v['events'] for k, v in sector_dict.items()])
    
    weighted = [v['events'] / total for k, v in sector_dict.items()]
    utils = [v['util'] for k, v in sector_dict.items()]
    
    final = 0
    for w, u in zip(weighted, utils):
        final += 1.0 * w * u
    return int(final)
        
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
def create_subs_sector_features(event_df):
    subs_events = event_df.withColumn(
            'total_subs_event', f.col('regular_event') + f.col('irregular_event')
    ).groupBy('hour', 'Subs_Id').agg(
        f.sum('regular_event').alias('regular_event_sum'),
        f.sum('irregular_event').alias('irregular_event_sum'),
        calc_weighted_utilisation(f.collect_list(f.struct('total_subs_event', 'sector_name', 'utilisation'))).alias('weighted_util')
    ).withColumn(
        'irregular_pct', 1.0 * f.col('irregular_event_sum') / (f.col('regular_event_sum') + f.col('irregular_event_sum'))
    )
    subs_events.persist()
    subs_events.show()
    
    sector_events = event_df.groupBy('hour', 'sector_name').agg(
        f.sum('regular_event').alias('regular_event_sum'),
        f.sum('irregular_event').alias('irregular_event_sum'),
        f.max('utilisation').alias('weighted_util')
    ).withColumn(
        'irregular_pct', 1.0 * f.col('irregular_event_sum') / (f.col('regular_event_sum') + f.col('irregular_event_sum'))
    )
    sector_events.persist()
    sector_events.show()
    
    return subs_events, sector_events

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
# subs_events.write.parquet(
#     's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/subs_event/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
# sector_events.write.parquet(
#     's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/sector_event/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Form Trace Event Dataset

In [133]:
# flat_event = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/flat_event/')
# flat_event = flat_event.withColumn(
#     'utilisation', f.col('utilisation').cast('int')
# )

# flat_event.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [148]:
# form trace_id selection
trace_ids = flat_event.select('hour', 'sector_name').dropDuplicates().sample(fraction=0.1).withColumn(
    'trace_id', f.row_number().over(Window.orderBy('hour', 'sector_name')) - 1
)
trace_ids.persist()
trace_ids.show()
trace_ids.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+--------+
|hour|         sector_name|trace_id|
+----+--------------------+--------+
|   0|200040_MacquarieS...|       0|
|   0|200046_StarCityHo...|       1|
|   0|200130_MGLBuilding_2|       2|
|   0| 200132_WavertonRS_1|       3|
|   0|200140_DarlingPar...|       4|
|   0|200219_AuroraPlace_4|       5|
|   0|200241_MMCDarling...|       6|
|   0|200247_WynyardNor...|       7|
|   0|200340_HumeThomas...|       8|
|   0|200359_CounselCha...|       9|
|   0|200424_PenrithPla...|      10|
|   0| 200433_WodenPlaza_1|      11|
|   0|    201489_Waratah_2|      12|
|   0|201520_HomebushBa...|      13|
|   0|201520_HomebushBa...|      14|
|   0|201529_SouthBlack...|      15|
|   0|201543_GlebeIsland_2|      16|
|   0|  201546_CrowsNest_1|      17|
|   0|201548_M5EastFrwa...|      18|
|   0|201563_UltimoSouth_1|      19|
+----+--------------------+--------+
only showing top 20 rows

21849

In [149]:
trace_event = flat_event.join(
    trace_ids,
    on=['hour', 'sector_name']
)
trace_event.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

79667109

In [151]:
# trace_event.write.mode('overwrite').parquet(
#     's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/trace_event/')

trace_event = spark.read.parquet(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/trace_event/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
subs_events, sector_events = create_subs_sector_features(trace_event)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+-----------------+-------------------+-------------+-------------+
|hour|  Subs_Id|regular_event_sum|irregular_event_sum|weighted_util|irregular_pct|
+----+---------+-----------------+-------------------+-------------+-------------+
|   0|280960963|                9|                  0|           49|          0.0|
|   0|281339172|                2|                  0|            9|          0.0|
|   0|281473592|               20|                  0|           76|          0.0|
|   0|282595650|                3|                  0|            8|          0.0|
|   0|282874909|               15|                  0|           22|          0.0|
|   0|284076669|                1|                  0|            7|          0.0|
|   0|284281456|                1|                  0|           87|          0.0|
|   0|318006680|                9|                  0|           24|          0.0|
|   0|318019813|               13|                  0|           11|          0.0|
|   

In [165]:
@f.udf(returnType=ArrayType(ArrayType(StringType())))
def construct_nodes(edge_input):
    """
    node types:
    0 - sector
    1 - user
    2 - cell (deprecated)
    """
    node_list = []
    for a in edge_input:
        if [a[0], 0] not in node_list:
            node_list.append([a[0], 0])
        if [a[1], 1] not in node_list:
            node_list.append([a[1], 1])
    return node_list

node_overview = trace_event.groupBy('trace_id', 'hour').agg(
    construct_nodes(
            f.collect_list(f.struct('sector_name', 'Subs_Id'))
        ).alias('node_list')
).withColumn(
    'node_id_type', f.explode('node_list')
).drop('node_list').withColumn(
    'node_name', f.col('node_id_type')[0]
).withColumn(
    'node_type', f.col('node_id_type')[1]
).drop('node_id_type').withColumn(
    'node_id', f.row_number().over(Window.partitionBy('trace_id').orderBy('node_type')) - 1
)

# node_overview = spark.read.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/node_overview/')
node_overview.persist()
node_overview.show()
node_overview.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----+-----------------+---------+-------+
|trace_id|hour|        node_name|node_type|node_id|
+--------+----+-----------------+---------+-------+
|     148|   0|202807_Woolwich_2|        0|      0|
|     148|   0|        387517594|        1|      1|
|     148|   0|        391325900|        1|      2|
|     148|   0|        382787677|        1|      3|
|     148|   0|        382858001|        1|      4|
|     148|   0|        369011404|        1|      5|
|     148|   0|        381185361|        1|      6|
|     148|   0|        318425547|        1|      7|
|     148|   0|        389517181|        1|      8|
|     148|   0|        366231684|        1|      9|
|     148|   0|        386932403|        1|     10|
|     148|   0|        364397606|        1|     11|
|     148|   0|        389641382|        1|     12|
|     148|   0|        364120337|        1|     13|
|     148|   0|        378697619|        1|     14|
|     148|   0|        386512709|        1|     15|
|     148|  

In [166]:
node_overview.write.mode('overwrite').parquet(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/node_overview/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
trace_event.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+---------+--------------------+-------------+-------------+---------------+-----------+---------------+--------+
|hour|         sector_name|  Subs_Id|           cell_name|event_type_cd|regular_event|irregular_event|utilisation|          state|trace_id|
+----+--------------------+---------+--------------------+-------------+-------------+---------------+-----------+---------------+--------+
|  12|227822_CanleyHeig...|367008741|227822_CanleyHeig...|            8|            0|              1|         38|New South Wales|   11597|
|  12|227822_CanleyHeig...|367008741|227822_CanleyHeig...|            7|            1|              0|         38|New South Wales|   11597|
|  12|227822_CanleyHeig...|367008741|227822_CanleyHeig...|            8|            0|              1|         38|New South Wales|   11597|
|  12|227822_CanleyHeig...|367008741|227822_CanleyHeig...|            8|            0|              1|         38|New South Wales|   11597|
|  12|227822_CanleyH

In [159]:
subs_events.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+-----------------+-------------------+-------------+-------------+
|hour|  Subs_Id|regular_event_sum|irregular_event_sum|weighted_util|irregular_pct|
+----+---------+-----------------+-------------------+-------------+-------------+
|   0|280960963|                9|                  0|           49|          0.0|
|   0|281339172|                2|                  0|            9|          0.0|
|   0|281473592|               20|                  0|           76|          0.0|
|   0|282595650|                3|                  0|            8|          0.0|
|   0|282874909|               15|                  0|           22|          0.0|
|   0|284076669|                1|                  0|            7|          0.0|
|   0|284281456|                1|                  0|           87|          0.0|
|   0|318006680|                9|                  0|           24|          0.0|
|   0|318019813|               13|                  0|           11|          0.0|
|   

In [170]:
subs_features = node_overview.join(
    subs_events,
    on=(node_overview.node_name==subs_events.Subs_Id) & (node_overview.hour==sector_events.hour),
).drop('hour', 'Subs_Id')
subs_features.show()
subs_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+-------+-----------------+-------------------+-------------+-------------+
|trace_id|node_name|node_type|node_id|regular_event_sum|irregular_event_sum|weighted_util|irregular_pct|
+--------+---------+---------+-------+-----------------+-------------------+-------------+-------------+
|     105|280960963|        1|      7|                9|                  0|           49|          0.0|
|     159|281339172|        1|    467|                2|                  0|            9|          0.0|
|     890|281473592|        1|    361|               20|                  0|           76|          0.0|
|     289|282595650|        1|      6|                3|                  0|            8|          0.0|
|     742|282874909|        1|     91|               15|                  0|           22|          0.0|
|     513|284076669|        1|     62|                1|                  0|            7|          0.0|
|     950|284281456|        1|    258|                1

In [171]:
sector_features = node_overview.join(
    sector_events,
    on=(node_overview.node_name==sector_events.sector_name) & (node_overview.hour==sector_events.hour),
).drop('hour', 'sector_name')
sector_features.show()
sector_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------+-------+-----------------+-------------------+-------------+--------------------+
|trace_id|           node_name|node_type|node_id|regular_event_sum|irregular_event_sum|weighted_util|       irregular_pct|
+--------+--------------------+---------+-------+-----------------+-------------------+-------------+--------------------+
|     148|   202807_Woolwich_2|        0|      0|             3863|                639|           13| 0.14193691692581076|
|     463|338006_BayswaterE...|        0|      0|              724|                 88|            4| 0.10837438423645321|
|     471|338029_PakenhamHe...|        0|      0|             1807|                 62|            5| 0.03317281968967362|
|     496|338334_BalaclavaE...|        0|      0|             2420|                172|           22| 0.06635802469135803|
|     833|588149_ModburyHei...|        0|      0|             1887|                261|           20| 0.12150837988826815|
|    1088|202661

In [172]:
node_features = subs_features.union(sector_features)
node_features.persist()
node_features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10423149

In [173]:
node_features.write.parquet('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/node_features/')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
node_features.filter('trace_id = 0').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+-------+-----------------+-------------------+-------------+------------------+
|trace_id|node_name|node_type|node_id|regular_event_sum|irregular_event_sum|weighted_util|     irregular_pct|
+--------+---------+---------+-------+-----------------+-------------------+-------------+------------------+
|       0|280226019|        1|     14|               13|                  0|            2|               0.0|
|       0|385704537|        1|      1|                6|                 14|            2|               0.7|
|       0|379370868|        1|     25|                9|                  0|            2|               0.0|
|       0|391102248|        1|     28|                4|                  0|            2|               0.0|
|       0|376264599|        1|     18|              121|                  0|            2|               0.0|
|       0|389548938|        1|      5|                2|                  0|            2|               0.0|
|       0|

## Output Node Feature Norm

In [181]:
assembler = VectorAssembler(inputCols=['regular_event_sum', 'irregular_event_sum', 'weighted_util', 'irregular_pct'], outputCol='feature')
scaler = MinMaxScaler(inputCol='feature', outputCol='feature_scaled')
pipeline = Pipeline(stages=[assembler, scaler])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [193]:
@f.udf(returnType=ArrayType(FloatType()))
def to_list(v):
    return [float(i) for i in v.values]

node_feature_norm = node_features.select(
    'trace_id', 'node_id', 'regular_event_sum', 'irregular_event_sum', 'weighted_util', 'irregular_pct')

node_feature_norm = pipeline.fit(node_feature_norm).transform(node_feature_norm).withColumn(
    'f', to_list('feature_scaled')
).select(['trace_id', 'node_id'] + [f.col('f')[i].alias(f'f_{i}') for i in range(4)])

node_feature_norm.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+------------+---+----------+---+
|trace_id|node_id|f_0         |f_1|f_2       |f_3|
+--------+-------+------------+---+----------+---+
|105     |7      |1.9184857E-4|0.0|0.4949495 |0.0|
|159     |467    |4.2633015E-5|0.0|0.09090909|0.0|
|890     |361    |4.2633014E-4|0.0|0.7676768 |0.0|
|289     |6      |6.394952E-5 |0.0|0.08080808|0.0|
|742     |91     |3.197476E-4 |0.0|0.22222222|0.0|
+--------+-------+------------+---+----------+---+
only showing top 5 rows

In [244]:
node_feature_norm = node_feature_norm.fillna(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [247]:
node_feature_norm.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/node_feature_norm/',
    header=True
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [246]:
node_feature_norm.filter('trace_id = 806').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+-----------+------------+---+----------+
|trace_id|node_id|        f_0|         f_1|f_2|       f_3|
+--------+-------+-----------+------------+---+----------+
|     806|      2|7.247613E-4|         0.0|0.0|       0.0|
|     806|      1|2.771146E-4|2.4335341E-4|0.0|0.23529412|
|     806|      0|9.805594E-4|2.4335341E-4|0.0|      0.08|
+--------+-------+-----------+------------+---+----------+

In [255]:
spark.read.csv('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/node_feature_norm/',
    header=True).filter('f_3 is NULL').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+---+---+---+---+
|trace_id|node_id|f_0|f_1|f_2|f_3|
+--------+-------+---+---+---+---+
+--------+-------+---+---+---+---+

In [241]:
node_features.filter('trace_id = 1728').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------+-------+-----------------+-------------------+-------------+-------------+
|trace_id|           node_name|node_type|node_id|regular_event_sum|irregular_event_sum|weighted_util|irregular_pct|
+--------+--------------------+---------+-------+-----------------+-------------------+-------------+-------------+
|    1728|           391083123|        1|      2|                2|                  0|            0|          0.0|
|    1728|           391866260|        1|      9|                5|                  0|            1|          0.0|
|    1728|           391709480|        1|      1|                3|                  0|           10|          0.0|
|    1728|           388384439|        1|      7|                1|                  0|            0|          0.0|
|    1728|           390033164|        1|      3|                7|                  1|            2|        0.125|
|    1728|           390411714|        1|     10|                2|     

### Output Edge Index

In [198]:
trace_event.show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+---------+---------------------------+-------------+-------------+---------------+-----------+---------------+--------+
|hour|sector_name           |Subs_Id  |cell_name                  |event_type_cd|regular_event|irregular_event|utilisation|state          |trace_id|
+----+----------------------+---------+---------------------------+-------------+-------------+---------------+-----------+---------------+--------+
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|8            |0            |1              |38         |New South Wales|11597   |
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|7            |1            |0              |38         |New South Wales|11597   |
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|8            |0            |1              |38         |New South Wales|11597   |
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|8            |0            |1          

In [251]:
edge_index = trace_event.groupBy('trace_id').agg(
        construct_edge_index(
            f.collect_list(f.struct('sector_name', 'Subs_Id'))
        ).alias('edge_index_list')
).withColumn(
    'edge_pair', f.explode('edge_index_list')
).drop('edge_index_list').withColumn(
    'src_name', f.col('edge_pair')[0]
).withColumn(
    'dst_name', f.col('edge_pair')[1]
).drop('edge_pair').join(
    node_overview.select(
        'trace_id', f.col('node_name').alias('src_name'), f.col('node_id').alias('src_id')
    ),
    on=['trace_id', 'src_name']
).join(
    node_overview.select(
        'trace_id', f.col('node_name').alias('dst_name'), f.col('node_id').alias('dst_id')
    ),
    on=['trace_id', 'dst_name']
).drop('dst_name', 'src_name')

edge_index.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+
|trace_id|src_id|dst_id|
+--------+------+------+
|148     |0     |406   |
|148     |0     |405   |
|148     |0     |404   |
|148     |0     |403   |
|148     |0     |402   |
+--------+------+------+
only showing top 5 rows

In [252]:
# adding self loop
self_loop = edge_index.select('trace_id', 'src_id').dropDuplicates().withColumn('dst_id', f.col('src_id'))
self_loop.show()
edge_index = edge_index.union(self_loop)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+
|trace_id|src_id|dst_id|
+--------+------+------+
|     148|     0|     0|
|     463|     0|     0|
|     471|     0|     0|
|     496|     0|     0|
|     833|     0|     0|
|    1088|     0|     0|
|    1238|     0|     0|
|    1342|     0|     0|
|    1580|     0|     0|
|    1591|     0|     0|
|    1645|     0|     0|
|    1829|     0|     0|
|    1959|     0|     0|
|    2122|     0|     0|
|    2142|     0|     0|
|    2366|     0|     0|
|    2659|     0|     0|
|    2866|     0|     0|
|    3175|     0|     0|
|    3749|     0|     0|
+--------+------+------+
only showing top 20 rows

In [253]:
edge_index.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10423149

In [254]:
edge_index.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/edge_index/', header=True
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [231]:
# check edge_index
edge_index = spark.read.csv('s3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/edge_index/', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [233]:
edge_index.filter('dst_id = 0').dropDuplicates().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+
|trace_id|src_id|dst_id|
+--------+------+------+
+--------+------+------+

### Output Node Types

In [207]:
node_types = node_overview.groupBy('trace_id').agg(
    construct_node_types(
        f.collect_list(f.struct('node_type', 'node_id'))
    ).alias('node_types')
).orderBy('trace_id')

node_types.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|trace_id|node_types                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
+--------+--------

In [208]:
node_types.withColumn('node_types', f.col('node_types').cast('string')).repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/node_types/', header=False
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Trace Info

In [205]:
trace_event.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+---------+---------------------------+-------------+-------------+---------------+-----------+---------------+--------+
|hour|sector_name           |Subs_Id  |cell_name                  |event_type_cd|regular_event|irregular_event|utilisation|state          |trace_id|
+----+----------------------+---------+---------------------------+-------------+-------------+---------------+-----------+---------------+--------+
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|8            |0            |1              |38         |New South Wales|11597   |
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|7            |1            |0              |38         |New South Wales|11597   |
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|8            |0            |1              |38         |New South Wales|11597   |
|12  |227822_CanleyHeights_1|367008741|227822_CanleyHeights_L18A_1|8            |0            |1          

In [215]:
trace_info = trace_event.select(
    'trace_id', 'utilisation'
).dropDuplicates().orderBy('trace_id').withColumn(
    'trace_bool', f.col('utilisation') < 75
).withColumn(
    'error_trace_type', f.col('trace_bool')
)
trace_info.show(5, False)
trace_info.groupBy('trace_bool').count().show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+----------+----------------+
|trace_id|utilisation|trace_bool|error_trace_type|
+--------+-----------+----------+----------------+
|0       |2          |true      |true            |
|1       |4          |true      |true            |
|2       |1          |true      |true            |
|3       |6          |true      |true            |
|4       |2          |true      |true            |
+--------+-----------+----------+----------------+
only showing top 5 rows

+----------+-----+
|trace_bool|count|
+----------+-----+
|true      |20973|
|false     |876  |
+----------+-----+

In [218]:
trace_info.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/trace_info/', header=True
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [222]:
train_ids = trace_info.filter('trace_bool = True').sample(fraction=0.6)
train_ids.groupBy('trace_bool').count().show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|trace_bool|count|
+----------+-----+
|true      |12619|
+----------+-----+

In [225]:
rest_trace_info = trace_info.join(
    train_ids.select('trace_id'),
    on='trace_id',
    how='left_anti'
)
rest_trace_info.groupBy('trace_bool').count().show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|trace_bool|count|
+----------+-----+
|true      |8354 |
|false     |876  |
+----------+-----+

In [226]:
eval_ids = rest_trace_info.sample(fraction=0.5)
eval_ids.groupBy('trace_bool').count().show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|trace_bool|count|
+----------+-----+
|true      |4175 |
|false     |436  |
+----------+-----+

In [227]:
test_ids = rest_trace_info.join(
    eval_ids.select('trace_id'),
    on='trace_id',
    how='left_anti'
)
test_ids.groupBy('trace_bool').count().show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|trace_bool|count|
+----------+-----+
|true      |4179 |
|false     |440  |
+----------+-----+

In [228]:
train_ids.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/train/', header=False
)

eval_ids.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/eval/', header=False
)

test_ids.repartition(1).write.mode('overwrite').csv(
    's3://kl-prod-tpgt-knowledge-lake-sandpit/TempDir/tmp/case_study/dataset_sector/csv/test/', header=False
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…